# Deep Learning - MNIST Classification Using Pytorch

In [ ]:
mnist_trainset = datasets.MNIST(root='./', train=True, download=True, transform=None)

# Mounting Drive and Unzipping MNIST Data



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp /content/drive/MyDrive/Sanwal_MSCS_20004/MNIST_Data.zip ./

!unzip MNIST_Data.zip

# Importing Libraries

In [2]:
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import plot_confusion_matrix, f1_score
import seaborn as sns
import pandas as pd
import time

# Loading Dataset and Applying Transforms

In [4]:
def loadDataset(data_dir,sizeTrain,sizeTest, batchSize,shuffle):

  # data_dir = 'drive/MyDrive' #DOWNLOAD DATA FOR THIS

  # TODO: Define transforms for the training data and testing data
  train_transforms = transforms.Compose([transforms.Resize((sizeTrain,sizeTrain)),transforms.ToTensor(),
                                        transforms.Normalize((0.0,),(0.5,)),
                                        transforms. Grayscale ( num_output_channels=1 )])
  
#   train_transforms = transforms.Compose([transforms.Resize((sizeTrain,sizeTrain)),transforms.ToTensor(),
#                                         transforms. Grayscale ( num_output_channels=1 )])

  test_transforms = transforms.Compose([transforms.Resize((sizeTest,sizeTest)),transforms.ToTensor(),
                                        transforms.Normalize((0.0,),(0.5,)),
                                        transforms. Grayscale ( num_output_channels=1 )])
  
#   test_transforms = transforms.Compose([transforms.Resize((sizeTest,sizeTest)),transforms.ToTensor(),
#                                         transforms. Grayscale ( num_output_channels=1 )])
  # Loading dataset using ImageFolder method. Pass transforms in here.
  
  train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
  test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

  # Here train and test dataloader are created using train_data and test_data objects. Batch size tells how many images are 
  # loaded for one iteration. 

  train_data, validation_data = torch.utils.data.random_split(train_data, [50000,10000])

  train_loader = torch.utils.data.DataLoader(train_data, batch_size=batchSize, shuffle=shuffle)
  test_loader = torch.utils.data.DataLoader(test_data, batch_size=batchSize,shuffle= shuffle)
  validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=batchSize,shuffle= shuffle)
  return train_loader, test_loader, validation_loader


# Creating Neural Network

In [5]:
class Net(nn.Module):
    def __init__(self,no_of_layers,input_dim,neurons_per_layer,dropout):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_dim,neurons_per_layer[0])
        self.fc2 = nn.Linear(neurons_per_layer[0], neurons_per_layer[1])
        self.dropout = nn.Dropout(dropout, inplace= True)
        self.fc3 = nn.Linear(neurons_per_layer[1],neurons_per_layer[2])
        # self.fc4 = nn.Linear(neurons_per_layer[2],neurons_per_layer[3])


    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        # x = self.fc4(x)
        return F.log_softmax(x,dim=1)

# Training Model

In [6]:
def train_model(data_size, model, train, validation, learning_rate, epochs, optimizer, loss_func, device):

  train_loss = []
  train_acc = []
  validate_acc = []
  validate_loss = []

  for epoch in range(epochs):

    total_tr_loss = 0
    correct = 0
    for batch_idx, (data1, target1) in enumerate(train):
        data1, target1 = Variable(data1), Variable(target1)
        data1, target1 = data1.to(device), target1.to(device)        
        data1 = data1.view(-1, data_size)
        optimizer.zero_grad()
        net_out = model(data1)
        loss = loss_func(net_out, target1)      
        loss.backward()
        optimizer.step()
        pred = net_out.data.max(1)[1]  # get the index of the max log-probability
        correct += pred.eq(target1.data).sum()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data1), len(train.dataset),
                          100. * batch_idx / len(train), loss.item()))
            total_tr_loss += loss.item()
    train_acc.append(correct / len(train.dataset))
    # train_loss.append(loss.item()
    train_loss.append(total_tr_loss)

    print('\nTrain Set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
                total_tr_loss, correct, len(train.dataset),
                100. * correct / len(train.dataset)))

    val_loss,val_accuracy = validate_model(model, validation, device, loss_func, data_size)
    validate_acc.append(val_accuracy)
    validate_loss.append(val_loss)
    
    torch.save(model.state_dict(), "./weights.pth")

  return train_acc, train_loss, validate_acc, validate_loss


# Validate Model

In [7]:
def validate_model(model,val_data,device,loss_func, data_size):

# this code cell calculates accuracy of test data on the trained model.
    validation_loss= 0
    correct = 0

    with torch.no_grad():
        for data, target in val_data:
            data, target = Variable(data), Variable(target)
            data, target = data.to(device), target.to(device)

            data = data.view(-1, data_size)
            net_out = model(data)
            # sum up batch loss
            validation_loss += loss_func(net_out, target).item()

            pred = net_out.data.max(1)[1]  # get the index of the max log-probability
            correct += pred.eq(target.data).sum()

        validation_loss /= len(val_data.dataset)
        validation_accuracy = correct/len(val_data.dataset)
        print('\nValidation Set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
                validation_loss, correct, len(val_data.dataset),
                100. * correct / len(val_data.dataset)))
        
    return validation_loss, validation_accuracy


# Plotting Train Accuracy and Loss, Validation Accuracy and Loss

In [8]:
def plot_train_result(training_accuracy,training_loss,validation_accuracy,validation_loss, batch_size, learning_rate, epochs, image_size):
  
  plot_list = []
  plot_list.append(training_accuracy)
  plot_list.append(validation_accuracy)
  plot_list.append(training_loss)
  plot_list.append(validation_loss)

  title_names = ["Training_accuracy", "Validation_accuracy","Training_loss","Validation_loss"]

  fig = plt.figure(figsize=(12,12))
  
  for i in range(len(plot_list)):
    plt.subplot(2,2,i+1)
    # plt.tight_layout()
    plt.title(title_names[i],fontweight="bold")
    plt.xlabel("Epochs")
    if i < 2:
      plt.ylabel("Accuracy")
    else:
      plt.ylabel("Loss")

    
    plt.plot(plot_list[i])
  fig.suptitle("\n Batch_Size:{}, Learning_Rate:{}, Epochs:{}, Image_Size:{}\n".format
                (batch_size, learning_rate, epochs, image_size), fontsize=14, fontweight="bold")
  
  plt.savefig("./train_result.png")

  plt.show()

  


# Performing Predicition on Trained Model

In [9]:
def perform_pred(model, test, device, batch_size, data_size, test_data_size):

  true_labels = []
  predicted_labels = []

  examples = enumerate(test)
  batch_idx, (example_data, example_targets) = next(examples)

  example_data, example_targets = example_data.to(device), example_targets.to(device)
  
  with torch.no_grad():

    example_data = example_data.view(-1, data_size)
    
    net_out = model(example_data)

  example_data = example_data.cpu().reshape(batch_size,1,test_data_size,test_data_size)

  fig = plt.figure()

  for i in range(6):

    plt.subplot(2,3,i+1)
    plt.tight_layout()
    plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  
    true_label = example_targets[i].item()
    true_labels.append(true_label)
  
    predicted_label = net_out.data.max(1, keepdim=True)[1][i].item()
    predicted_labels.append(predicted_label)

    plt.title("Ground-Truth: {}\n Prediction: {}".format(true_label, predicted_label ),
              fontweight="bold")
  
    plt.xticks([])
    plt.yticks([])
  plt.savefig("./prediction.png")


  
  return true_labels, predicted_labels

# Testing Model

In [10]:
def test_model(model,test_data, device,loss_func, batch_size, data_size, test_data_size):

# this code cell calculates accuracy of test data on the trained model.
  test_loss = 0
  correct = 0
  target_list = []
  pred_list = []
  with torch.no_grad():
      for data, target in test_data:
          data, target = Variable(data), Variable(target)
          data, target = data.to(device), target.to(device)

          data = data.view(-1, data_size)
          net_out = model(data)
          # sum up batch loss
          test_loss += loss_func(net_out, target).item()

          pred = net_out.data.max(1)[1]  # get the index of the max log-probability
          
          
          correct += pred.eq(target.data).sum()

          # cm = confusion_matrix(target.cpu(),pred.cpu())
          target_list.extend( list(target.cpu()) )
          pred_list.extend( list(pred.cpu()) )
      test_loss /= len(test_data.dataset)
      validation_accuracy = correct/len(test_data.dataset)
      print('\nTest Set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
              test_loss, correct, len(test_data.dataset),
              100. * correct / len(test_data.dataset)))
      
      target, prediction = perform_pred(model,test_data, device, batch_size, data_size, test_data_size)

  return target, prediction, pred_list, target_list




# Calculating and Plotting F1_Score and Confusion_Matrix

In [11]:
def calculate_plot_cm_f1(pred_list, target_list):
  
  cm = confusion_matrix(pred_list,target_list)
  f1_scr = f1_score(pred_list, target_list, average='weighted')
  print("F1-Score : ",f1_scr)
  columns = [ str(i) for i in range(10) ]
  cm_df = pd.DataFrame(cm,columns,columns)  
  fig = plt.figure(figsize=(12,5)) 
  sns.heatmap(cm_df, annot=True)
  plt.ylabel("True Labels") 
  plt.xlabel("Predicted Labels")
  fig.suptitle("Confusion_Matrix", fontweight="bold")
  plt.savefig("confusion.png")
  plt.show()

 


# Single Perform_MNIST_NN() Function Calling All Functions()

In [12]:
def Perform_MNIST_NN():

  # starting time
  start = time.time()

  print(start)


  test_data_size = 28
  train_data_size = 28
  batch_size = 32
  shuffle = True

  # Load Datasets
  train,test,validation = loadDataset('/content', test_data_size, train_data_size, batch_size, shuffle )

  # Build neural network
  number_of_hidden_layers = 2
  input_layer_dim = 28*28
  neurons_per_layer = [128,64,10]
  drop_out = 0.9

  model = Net(number_of_hidden_layers, input_layer_dim, neurons_per_layer, drop_out)

  # this statement tell our code if there gpu available on our machine or not.
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model.to(device)
  print(model)
  print(device)

  # Initializing parameters
  data_size = 28*28
  learning_rate=0.01
  epochs= 10
  optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.5)
#   optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas = (0.9,0.999))
  loss_func = nn.NLLLoss()
#   loss_func = nn.CrossEntropyLoss()
#   loss_func = nn.KLDivLoss()

  #Training the model
  training_accuracy, training_loss, validation_accuracy, validation_loss = train_model(data_size, model, train, 
                                                validation, learning_rate, epochs, optimizer, loss_func, device)

  #Plotting the accuracy and loss of train, validation data
  plot_train_result(training_accuracy, training_loss, validation_accuracy, validation_loss, batch_size, 
                    learning_rate, epochs, test_data_size)

  #Testing the model and perform predictions
  
  model.load_state_dict(torch.load("./weights.pth"))
  target, prediction, pred_list, target_list = test_model(model,test, device, loss_func, batch_size, data_size, test_data_size)

  print("True Label: {} \nPrediction: {}".format(target, prediction ))

  calculate_plot_cm_f1(pred_list, target_list)

  end = time.time()
  print(f"Runtime of the program is {end - start}")
  return model




# Calling Perform_MNIST_NN() Function

In [ ]:
model = Perform_MNIST_NN()

# Loading Saved Model and Testing, Predicting

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss_func = nn.CrossEntropyLoss()
test_data_size = 28
train_data_size = 28
batch_size = 64
shuffle = True
  # Load Datasets
train,test,validation = loadDataset('/content', test_data_size, train_data_size, batch_size, shuffle )

model.load_state_dict(torch.load("./weights.pth"))
target, prediction, pred_list, target_list = test_model(model,test, device, loss_func)

print("True Label: {} \nPrediction: {}".format(target, prediction ))

calculate_plot_cm_f1(pred_list, target_list)